In [14]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key


In [15]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Kaspiyskiy,RU,2020-10-05 00:34:11,45.39,47.37,60.42,63,94,14.54
1,1,Faanui,PF,2020-10-05 00:34:12,-16.48,-151.75,78.40,78,67,13.38
2,2,Poncitlan,MX,2020-10-05 00:34:12,20.37,-102.92,78.80,26,5,3.80
3,3,Atuona,PF,2020-10-05 00:34:12,-9.80,-139.03,77.83,78,26,15.75
4,4,Vaini,TO,2020-10-05 00:34:12,-21.20,-175.20,75.20,88,75,12.75


In [25]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [40]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height=&#39;420px&#39;))

In [41]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


In [42]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Faanui,PF,2020-10-05 00:34:12,-16.48,-151.75,78.40,78,67,13.38
2,2,Poncitlan,MX,2020-10-05 00:34:12,20.37,-102.92,78.80,26,5,3.80
3,3,Atuona,PF,2020-10-05 00:34:12,-9.80,-139.03,77.83,78,26,15.75
4,4,Vaini,TO,2020-10-05 00:34:12,-21.20,-175.20,75.20,88,75,12.75
13,13,Kapaa,US,2020-10-05 00:34:14,22.08,-159.32,86.00,62,40,11.41
14,14,Bengkulu,ID,2020-10-05 00:30:13,-3.80,102.27,78.51,83,100,7.87
26,26,Acajutla,SV,2020-10-05 00:32:46,13.59,-89.83,80.02,74,100,3.44
27,27,Balaipungut,ID,2020-10-05 00:34:17,1.05,101.28,77.00,94,9,1.54
29,29,Georgetown,MY,2020-10-05 00:30:12,5.41,100.34,80.60,88,20,5.82
38,38,Hilo,US,2020-10-05 00:34:20,19.73,-155.09,86.00,51,1,11.41


In [43]:
preferred_cities_df.count()


City_ID       176
City          176
Country       176
Date          176
Lat           176
Lng           176
Max Temp      176
Humidity      176
Cloudiness    176
Wind Speed    176
dtype: int64

In [44]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Faanui,PF,78.40,-16.48,-151.75,
2,Poncitlan,MX,78.80,20.37,-102.92,
3,Atuona,PF,77.83,-9.80,-139.03,
4,Vaini,TO,75.20,-21.20,-175.20,
13,Kapaa,US,86.00,22.08,-159.32,
14,Bengkulu,ID,78.51,-3.80,102.27,
26,Acajutla,SV,80.02,13.59,-89.83,
27,Balaipungut,ID,77.00,1.05,101.28,
29,Georgetown,MY,80.60,5.41,100.34,
38,Hilo,US,86.00,19.73,-155.09,


In [45]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Faanui,PF,78.40,-16.48,-151.75,
2,Poncitlan,MX,78.80,20.37,-102.92,
3,Atuona,PF,77.83,-9.80,-139.03,
4,Vaini,TO,75.20,-21.20,-175.20,
13,Kapaa,US,86.00,22.08,-159.32,
14,Bengkulu,ID,78.51,-3.80,102.27,
26,Acajutla,SV,80.02,13.59,-89.83,
27,Balaipungut,ID,77.00,1.05,101.28,
29,Georgetown,MY,80.60,5.41,100.34,
38,Hilo,US,86.00,19.73,-155.09,


In [46]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


In [38]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.


KeyboardInterrupt: 

In [37]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height=&#39;420px&#39;))

In [ ]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig


In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


In [32]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height=&#39;420px&#39;))